# Guppy Demo

This notebook gives a brief introduction to Guppy.

Make sure that you have installed Guppy via `pip install guppylang`.

In [1]:
import guppylang

## Getting Started

Guppy programs are defined within a `GuppyModule`.

In [2]:
from guppylang import GuppyModule

module = GuppyModule(name="example_module")

Guppy functions look like regular Python functions that are annotated with the `@guppy` decorator.

In [3]:
from guppylang import guppy

@guppy(module)
def factorial(x: int) -> int:
    """Classical factorial function in Guppy."""
    acc = 1
    while x > 0:
        acc *= x
        x -= 1
    return acc

Guppy provides a special `qubit` type that quantum operations act on. Quantum operations become available once we import and load the `guppylang.std.quantum` module.

In [4]:
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, measure, h, cx

module.load(qubit, h, cx, measure)

@guppy(module)
def bell() -> bool:
    """Prepares and measures a bell state.
    
    Returns the parity of the measurement results.
    """
    # Allocate two fresh qubits
    q1, q2 = qubit(), qubit()
    # Entangle
    h(q1)
    cx(q1, q2)
    h(q2)
    # Measure
    b1, b2 = measure(q1), measure(q2)
    return b1 ^ b2

Once all functions have been defined, the module can be compiled.

In [5]:
program = module.compile()

Note that execution of programs is still work-in-progress and therefore not included in this notebook.

## Type System

Guppy is strongly typed and requires type annotations for function arguments and returns as in the examples above.

Ill-typed programs are rejected by the compiler. For example, observe the error when trying to compile the program below.

In [6]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(x: int) -> int:
    # Try to add a tuple to an int
    return x + (x, x)

bad_module.compile()  # Raises an error

Error: Operator not defined (at <In [6]>:6:11)
  | 
4 | def bad(x: int) -> int:
5 |     # Try to add a tuple to an int
6 |     return x + (x, x)
  |            ^^^^^^^^^^ Binary operator `+` not defined for `int` and `(int, int)`

Guppy compilation failed due to 1 previous error


Furthermore, the Guppy compiler ensures that variables are definitely initialised when they are used.

In [7]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(b: bool) -> int:
    if b:
        x = 4
    return x  # x not defined if b is False

bad_module.compile()  # Raises an error

Error: Variable not defined (at <In [7]>:7:11)
  | 
5 |     if b:
6 |         x = 4
7 |     return x  # x not defined if b is False
  |            ^ `x` might be undefined ...
  | 
5 |     if b:
  |        - ... if this expression is `False`

Guppy compilation failed due to 1 previous error


Similarly, variables must have a unique type when they are used.

In [8]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def bad(b: bool) -> int:
    if b:
        x = 4
    else:
        x = True
    return int(x)  # x has different types depending on b

bad_module.compile()  # Raises an error

Error: Different types (at <In [8]>:9:15)
  | 
7 |     else:
8 |         x = True
9 |     return int(x)  # x has different types depending on b
  |                ^ Variable `x` may refer to different types
  | 
6 |         x = 4
  |         - This is of type `int`
  | 
8 |         x = True
  |         - This is of type `bool`

Guppy compilation failed due to 1 previous error


## Linear Types

Qubits in Guppy obey *linear typing*, i.e. they cannot be copied and may only be used once before they are reassigned.

In [9]:
bad_module = GuppyModule(name="bad_module")
bad_module.load_all(guppylang.std.quantum)

@guppy(bad_module)
def bad(q: qubit @owned) -> qubit:
    cx(q, q)
    return q

bad_module.compile()  # Raises an error

Error: Copy violation (at <In [9]>:6:10)
  | 
4 | @guppy(bad_module)
5 | def bad(q: qubit @owned) -> qubit:
6 |     cx(q, q)
  |           ^ Variable `q` with non-copyable type `qubit` cannot be
  |             borrowed ...
  | 
6 |     cx(q, q)
  |        - since it was already borrowed here

Guppy compilation failed due to 1 previous error


Similarly, qubits cannot be implicitly dropped. They need to be returned from functions or explicitly measured or discarded.

In [10]:
bad_module = GuppyModule(name="bad_module")
bad_module.load_all(guppylang.std.quantum)

@guppy(bad_module)
def bad(q: qubit @owned) -> qubit:
    tmp = qubit()
    cx(tmp, q)
    #discard(tmp)  # Compiler complains if tmp is not explicitly discarded
    return q

bad_module.compile()  # Raises an error

Error: Drop violation (at <In [10]>:6:4)
  | 
4 | @guppy(bad_module)
5 | def bad(q: qubit @owned) -> qubit:
6 |     tmp = qubit()
  |     ^^^ Variable `tmp` with non-droppable type `qubit` is leaked

Help: Make sure that `tmp` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error


## Mutual Recursion

Guppy functions support (mutual) recursion:

In [11]:
module = GuppyModule("mutual_recursion")

@guppy(module)
def is_even(x: int) -> bool:
    if x == 0:
        return True
    return is_odd(x-1)

@guppy(module)
def is_odd(x: int) -> bool:
    if x == 0:
        return False
    return is_even(x-1)

program = module.compile()

## Structs

Structs can be used to group quantum and classical data.

In [12]:
module = GuppyModule("structs")
module.load_all(guppylang.std.quantum)

@guppy.struct(module)
class QubitPair:
    q1: qubit
    q2: qubit

    @guppy(module)
    def method(self: "QubitPair") -> None:
        h(self.q1)
        cx(self.q1, self.q2)

@guppy(module)
def make_struct() -> QubitPair:
    pair = QubitPair(qubit(), qubit())
    pair.method()
    return pair

program = module.compile()

## Nested & Higher-Order Functions

Guppy supports nested function definitions with variables captured from outer scopes.

In [13]:
module = GuppyModule("nested_function")

@guppy(module)
def outer(x: int) -> int:
    def nested(y: int) -> int:
        return x + y
    return nested(42)

program = module.compile()

However, similar to regular Python, variables from enclosing functions scopes may not be modified. Python's `global` and `nonlocal` statements are not supported by Guppy.

In [14]:
bad_module = GuppyModule(name="bad_module")

@guppy(bad_module)
def outer(x: int) -> int:
    def nested() -> None:
        x += 1  # Mutation of captured variable x is not allowed
    nested()
    return x

bad_module.compile()  # Raises an error

Error: Illegal assignment (at <In [14]>:6:8)
  | 
4 | def outer(x: int) -> int:
5 |     def nested() -> None:
6 |         x += 1  # Mutation of captured variable x is not allowed
  |         ^^^^^^ Variable `x` may not be assigned to since `x` is captured
  |                from an outer scope
  | 
4 | def outer(x: int) -> int:
  |           ------ `x` defined here

Guppy compilation failed due to 1 previous error


Functions can be used as higher-order values using Python's `Callable` type.

In [15]:
from typing import Callable

module = GuppyModule("higher_order")

@guppy(module)
def increment_by(inc: int) -> Callable[[int], int]:
    def closure(x: int) -> int:
        return x + inc
    return closure

@guppy(module)
def main(x: int) -> int:
    inc5 = increment_by(5)
    return inc5(x)

program = module.compile()